In [1]:
using Compat
using MomentumBasedControl
using RigidBodyDynamics
using RigidBodyDynamics.Contact
using AtlasRobot

INFO: Recompiling stale cache file /home/twan/code/RigidBodyDynamics/lib/v0.6/MomentumBasedControl.ji for module MomentumBasedControl.


In [2]:
mechanism = AtlasRobot.mechanism()
remove_fixed_tree_joints!(mechanism);

In [3]:
# add environment
rootframe = root_frame(mechanism)
ground = HalfSpace3D(Point3D(rootframe, 0., 0., 0.), FreeVector3D(rootframe, 0., 0., 1.))
add_environment_primitive!(mechanism, ground);

In [4]:
const SparseSymmetric64 = Symmetric{Float64,SparseMatrixCSC{Float64,Int}}

struct StandingController{N}
    foottasks::Dict{Symbol, SpatialAccelerationTask}
    pelvistask::AngularAccelerationTask
    jointtasks::Dict{Joint{Float64, Revolute{Float64}}, Weighted{JointAccelerationTask{Revolute{Float64}}}}
    contactsettings::Dict{RigidBody{Float64}, Vector{ContactSettings{N}}}

    function StandingController{N}(
                mech::Mechanism,
                feet::Dict{Symbol, RigidBodyDynamics.RigidBody{Float64}},
                pelvis::RigidBody{Float64},
                contactinfo::Dict{RigidBody{Float64}, Vector{ContactInfo}}) where N
        world = root_body(mech)
        revolutejoints = filter(j -> joint_type(j) isa Revolute, tree_joints(mech))
        foottasks = Dict(side => SpatialAccelerationTask(mech, path(mech, world, foot)) for (side, foot) in feet)
        pelvistask = AngularAccelerationTask(mech, path(mech, world, pelvis))
        jointtasks = Dict(j => Weighted(JointAccelerationTask(mech, j), Symmetric(speye(1))) for j in revolutejoints)
        foreach(disable!, values(jointtasks))
        contactsettings = Dict(b => ContactSettings{N}.(infos) for (b, infos) in contactinfo)
        new(foottasks, pelvistask, jointtasks, contactsettings)
    end
end

In [5]:
function set_joint_accel_regularization!(controller::StandingController, weight::Number)
    for task in values(controller.jointtasks)
        task.weight.data[:] = weight
    end
    nothing
end

set_joint_accel_regularization! (generic function with 1 method)

In [6]:
feet = Dict(:left => findbody(mechanism, "l_foot"), :right => findbody(mechanism, "r_foot"))
pelvis = findbody(mechanism, "pelvis")
contactinfo = Dict{RigidBody{Float64}, Vector{ContactInfo}}()
μ = 0.8
for body in bodies(mechanism)
    points = RigidBodyDynamics.location.(RigidBodyDynamics.contact_points(body))
    if !isempty(points)
        normal = FreeVector3D(default_frame(body), 0.0, 0.0, 1.0)
        contactinfo[body] = [ContactInfo(point, normal, μ) for point in points]
    end
end
highlevelcontroller = StandingController{4}(mechanism, feet, pelvis, contactinfo);

In [7]:
using TypeSortedCollections

function motiontasks(controller::StandingController)
    ret = MotionTask[]
    append!(ret, values(controller.foottasks))
    push!(ret, controller.pelvistask)
    append!(ret, values(controller.jointtasks))
    TypeSortedCollection(ret)
end

contactsettings(controller::StandingController) = vcat(map(values, controller.contactsettings))

contactsettings (generic function with 1 method)

In [8]:
using OSQP.MathOptInterfaceOSQP
optimizer = OSQPOptimizer();

In [19]:
using SimpleQP
import SimpleQP.Functions: QuadraticFunction
import MomentumBasedControl: dimension, task_error

struct MomentumBasedController{O<:MathOptInterface.AbstractOptimizer, N, M}
    # dynamics-related
    centroidalframe::CartesianFrame3D
    result::DynamicsResult{Float64, Float64}
    momentummatrix::MomentumMatrix{Matrix{Float64}}
    wrenchmatrix::WrenchMatrix{Matrix{Float64}}

    # contact info and motion tasks
    motiontasks::M
    contactsettings::Dict{RigidBody{Float64}, Vector{ContactSettings{N}}}

    # qp-related
    qpmodel::SimpleQP.Model{O}

    function MomentumBasedController(
                mechanism::Mechanism{Float64},
                optimizer::O, 
                contactsettings::Dict{RigidBody{Float64}, Vector{ContactSettings{N}}},
                motiontasks::M) where {O<:MathOptInterface.AbstractOptimizer, N, M}
        nv = num_velocities(mechanism)

        # dynamics-related
        centroidalframe = CartesianFrame3D("centroidal")
        result = DynamicsResult(mechanism)
        A = MomentumMatrix(centroidalframe, zeros(3, nv), zeros(3, nv))
        wrenchmatrix = WrenchMatrix(centroidalframe, zeros(3, 0), zeros(3, 0))

        # qp-related
        qpmodel = SimpleQP.Model(optimizer)
        objective = QuadraticFunction()
    
        # joint accelerations
        v̇ = [Variable(qpmodel) for _ = 1 : nv]
    
        # motion tasks
        foreach(motiontasks) do task
            dim = dimension(task)
            if task isa Weighted
                e = [Variable(qpmodel) for _ = 1 : dim]
                objective += QuadraticTerm(task.weight, e)
                @constraint(qpmodel, task_error(task, v̇) == LinearTerm(eye(dim), e))
            else
                @constraint(qpmodel, task_error(task, v̇) == Constant(zeros(dim)))
            end
        end
    
#         # contact force multipliers
#         for (body, bodycontacts) in contactsettings
#             ρ
#         end
        
        # Newton-Euler
        fg = mass(mechanism) * mechanism.gravitational_acceleration
#         LinearTerm(A.angular, v̇) + Ȧv.angular == sum()

        new{O, N, M}(centroidalframe, result, A, wrenchmatrix, motiontasks, contactsettings, qpmodel)
    end
end

In [20]:
lowlevelcontroller = MomentumBasedController(mechanism, optimizer, highlevelcontroller.contactsettings, motiontasks(highlevelcontroller));

In [14]:
foreach(tasks)

LoadError: [91mUndefVarError: tasks not defined[39m

In [21]:
lowlevelcontroller.qpmodel.zeroconstraints

33-element Array{SimpleQP.AffineConstraint{MathOptInterface.Zeros},1}:
 SimpleQP.AffineConstraint{MathOptInterface.Zeros}(SimpleQP.DataPair{SimpleQP.Functions.AffineFunction,MathOptInterface.VectorAffineFunction{Float64}}(SimpleQP.Functions.AffineFunction(SimpleQP.Functions.Sum{SimpleQP.Functions.Scaled{SimpleQP.Functions.LinearTerm}}(SimpleQP.Functions.Scaled{SimpleQP.Functions.LinearTerm}[SimpleQP.Functions.Scaled{SimpleQP.Functions.LinearTerm}(1.0, SimpleQP.Functions.LinearTerm([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], SimpleQP.Functions.Variable[SimpleQP.Functions.Variable(1), SimpleQP.Functions.Variable(2), SimpleQP.Functions.Variable(3), SimpleQP.Functions.Variable(4), SimpleQP.Functions.Variable(5), SimpleQP.Functions.Variable(6), SimpleQP.Functions.Variable(7), SimpleQP.Functions.Variable(8), SimpleQP.Functions.Variable(9), SimpleQP.Functions.Variable(10)  …  SimpleQP.Functions.Variable(27), SimpleQP.Functions.Variable(28), SimpleQP.Funct